In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

## **Connection a selenium et extraction des noms de films**

In [3]:
# Chemin du ChromeDriver
driver_path = r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\chromedriver-win64\chromedriver-win64\chromedriver.exe"

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd



# Créer un objet Service
service = Service(driver_path)

# Initialiser les options du navigateur Chrome
options = Options()
options.add_argument("--headless")  # Lancer Chrome en mode sans tête (optionnel)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialiser le navigateur Chrome avec le service et les options
driver = webdriver.Chrome(service=service, options=options)

# Accéder à la page
base_url = "https://letterboxd.com/films/popular/this/week/page/"
page_number = 1
film_titles = []

# Fonction pour récupérer les films d'une page
def get_films():
    film_divs = driver.find_elements(By.CSS_SELECTOR, "div.react-component.poster.film-poster")
    for div in film_divs:
        title = div.get_attribute("data-film-name")
        if title:
            film_titles.append(title)

# Charger les films depuis plusieurs pages
clicks = 0
max_clicks = 50 # Charger 20 pages
while clicks < max_clicks:
    url = f"{base_url}{page_number}/"  # Générer l'URL pour chaque page
    driver.get(url)
    time.sleep(5)  # Attendre que la page se charge

    get_films()  # Extraire les films de la page actuelle
    print(f"Page {page_number} extraite.")

    page_number += 1  # Passer à la page suivante
    clicks += 1

# Fermer le navigateur
driver.quit()

# Sauvegarder les films dans un DataFrame
df_films = pd.DataFrame({"Film Title": film_titles})
df_films.to_csv("letterboxd_movies.csv", index=False)

print(f"{len(film_titles)} films extraits et enregistrés dans 'letterboxd_movies.csv'.")


Page 1 extraite.
Page 2 extraite.
Page 3 extraite.
Page 4 extraite.
Page 5 extraite.
Page 6 extraite.
Page 7 extraite.
Page 8 extraite.
Page 9 extraite.
Page 10 extraite.
Page 11 extraite.
Page 12 extraite.
Page 13 extraite.
Page 14 extraite.
Page 15 extraite.
Page 16 extraite.
Page 17 extraite.
Page 18 extraite.
Page 19 extraite.
Page 20 extraite.
Page 21 extraite.
Page 22 extraite.
Page 23 extraite.
Page 24 extraite.
Page 25 extraite.
Page 26 extraite.
Page 27 extraite.
Page 28 extraite.
Page 29 extraite.
Page 30 extraite.
Page 31 extraite.
Page 32 extraite.
Page 33 extraite.
Page 34 extraite.
Page 35 extraite.
Page 36 extraite.
Page 37 extraite.
Page 38 extraite.
Page 39 extraite.
Page 40 extraite.
Page 41 extraite.
Page 42 extraite.
Page 43 extraite.
Page 44 extraite.
Page 45 extraite.
Page 46 extraite.
Page 47 extraite.
Page 48 extraite.
Page 49 extraite.
Page 50 extraite.
3600 films extraits et enregistrés dans 'letterboxd_movies.csv'.


In [6]:
len(df_films["Film Title"].unique())

3472

## **Amelioration des noms pour pouvoir les utiliser dans les urls**

In [10]:
import pandas as pd
import re
from unidecode import unidecode

# Charger le fichier CSV
df = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\letterboxd_movies.csv")

# Fonction pour transformer les noms des films
def transformer_nom(film):

    film = film.replace(" ", "-")
    film = film.replace("–", "-")
    film = film.replace("&", "")
    film = film.replace("'", "")
    film = film.replace(".", "-")
    film = film.replace(",", "")
    film = film.replace("#", "").replace(":", "")
    film = film.replace("é", "e").replace("è", "e").replace("à", "a")
    film = re.sub(r'-+', '-', film)
    film = film.replace("***", "")
    film = film.replace("*", "")
    film = film.replace("(", "").replace(")", "")
    film = film.replace("!", "").replace("?", "").replace(":", "").replace(";", "").replace('"', "")
    film = re.sub(r'[^\w\s-]', '', film)
    film = film.replace('[', '').replace(']', '')
    film = unidecode(film)
    return film.lower()

# Appliquer la transformation
df['MOVIE_NAME'] = df['Film Title'].apply(transformer_nom)

# Sauvegarder dans un nouveau fichier CSV
df.to_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\movies_with_reviews.csv", index=False)

# Afficher les premières lignes pour vérification
print(df.head())


      Film Title     MOVIE_NAME
0       Carry-On       carry-on
1      Nosferatu      nosferatu
2     Home Alone     home-alone
3  The Holdovers  the-holdovers
4          Anora          anora


## **Creation des urls**

In [11]:
import pandas as pd

# Charger le fichier CSV existant
csv_file = r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\movies_with_reviews.csv"

# Charger les données
df = pd.read_csv(csv_file)

# Ajouter la colonne URL
base_url = "https://letterboxd.com/film/"
df['URL_FILM'] = base_url + df['MOVIE_NAME'] + "/"

# Mettre à jour le fichier CSV existant
df.to_csv(csv_file, index=False)

# Afficher les premières lignes pour vérification
print(df.head())


      Film Title     MOVIE_NAME                                    URL_FILM
0       Carry-On       carry-on       https://letterboxd.com/film/carry-on/
1      Nosferatu      nosferatu      https://letterboxd.com/film/nosferatu/
2     Home Alone     home-alone     https://letterboxd.com/film/home-alone/
3  The Holdovers  the-holdovers  https://letterboxd.com/film/the-holdovers/
4          Anora          anora          https://letterboxd.com/film/anora/


## **Extraction des reviews**

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Charger le fichier CSV existant
df = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\movies_with_reviews.csv")

def fetch_reviews(url):
    reviews_url = url + "reviews/by/activity/"
    response = requests.get(reviews_url)
    
    if response.status_code != 200:
        print(f"Failed to fetch {reviews_url}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extraire les critiques
    review_elements = soup.select('div.body-text p')
    reviews = [review.text.strip() for review in review_elements]
    
    return reviews

# Ajouter une colonne 'REVIEWS' et la remplir avec les critiques
df['REVIEWS'] = ""



In [ ]:
# Parcourir les lignes à partir de start_row
for idx, row in df.iloc[::].iterrows():    
    # Ajouter les critiques
    if pd.isna(row["REVIEWS"]) or row["REVIEWS"] == "":
        reviews = fetch_reviews(row['URL_FILM'])
        reviews_text = "**".join(reviews) if reviews else "Aucune critique disponible"
        df.loc[idx, "REVIEWS"] = reviews_text
    
    # Sauvegarder les résultats après chaque itération pour éviter les pertes
    df.to_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\last.csv", index=False)


print("Traitement terminé.")

Traitement terminé.


In [ ]:

df2 = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\movies_with_reviews.csv")
df1 = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\last.csv")
df = pd.concat([df1, df2], ignore_index=True)
df.drop_duplicates(subset=["MOVIE_NAME"], keep="first", inplace=True)

In [2]:
df.to_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\last.csv", index=False)

## **Extraction du synopsis**

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re


input_csv = r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\last.csv"  
df = pd.read_csv(input_csv)

# Ajouter les colonnes "Synopsis"
for col in ["Synopsis"]:
    if col not in df.columns:
        df[col] = None

# Fonction pour extraire le synopsis
def fetch_movie_details(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Échec lors de la récupération de {url}")
            return {"Synopsis": "Indisponible"}
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extraire le synopsis
        synopsis_div = soup.find('div', class_='review body-text -prose -hero prettify')
        if synopsis_div:
            synopsis_p = synopsis_div.find('p')
            synopsis = synopsis_p.text.strip() if synopsis_p else "Indisponible"
        else:
            synopsis = "Indisponible"

        return {"Synopsis": synopsis}
    
    except Exception as e:
        print(f"Erreur pour {url}: {e}")
        return {"Synopsis": "Indisponible"}


# Récupérer les détails pour chaque film à partir de l'indice spécifié
for idx, row in df.iloc[::].iterrows(): 
    if pd.notna(row["Synopsis"]):  # Passer les films déjà traités
        continue
    
    details = fetch_movie_details(row['URL_FILM'])
    df.loc[idx, "Synopsis"] = details["Synopsis"]

    # Sauvegarder les résultats après chaque itération pour éviter les pertes
    df.to_csv(input_csv, index=False)

print("Extraction terminée et sauvegardée dans le fichier CSV d'origine.")


Extraction terminée et sauvegardée dans le fichier CSV d'origine.


In [4]:
df1 = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\last.csv")
df2 = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\peur de perdre\last.csv")  

In [5]:
df1

,Film Title,MOVIE_NAME,URL_FILM,REVIEWS,Synopsis,Year,Cast,Duration,GENRES,COUNTRIES,PRODUCERS,Rating
0,Carry-On,carry-on,https://letterboxd.com/film/carry-on/,Aucune critique disponible,Indisponible,Inconnue,NaN,NaN,NaN,NaN,NaN,NaN
1,Nosferatu,nosferatu,https://letterboxd.com/film/nosferatu/,I only watched this to understand the Spongebo...,Indisponible,Inconnue,NaN,NaN,NaN,NaN,NaN,NaN
2,Home Alone,home-alone,https://letterboxd.com/film/home-alone/,Young Patrick Bateman**do you think the wet ba...,Indisponible,Inconnue,NaN,NaN,NaN,NaN,NaN,NaN
3,The Holdovers,the-holdovers,https://letterboxd.com/film/the-holdovers/,There's a handshake in this that feels more ov...,Indisponible,Inconnue,NaN,NaN,NaN,NaN,NaN,NaN
4,Anora,anora,https://letterboxd.com/film/anora/,A terrifying tale of dating a mama’s boy**Sean...,Indisponible,Inconnue,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3463,Evangelion: 3.0 You Can (Not) Redo,evangelion-3-0-you-can-not-redo,https://letterboxd.com/film/evangelion-3-0-you...,NaN,Indisponible,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3464,Hanna,hanna,https://letterboxd.com/film/hanna/,NaN,"Raised by her father, an ex-CIA agent, in the ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3465,Puppy Love,puppy-love,https://letterboxd.com/film/puppy-love/,NaN,When single mom Megan Nolan moves to a new tow...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3466,Hancock,hancock,https://letterboxd.com/film/hancock/,NaN,Hancock is a down-and-out superhero who's forc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df2

,Film Title,MOVIE_NAME,URL_FILM,REVIEWS
0,Carry-On,carry-on,https://letterboxd.com/film/carry-on/,Aucune critique disponible
1,Nosferatu,nosferatu,https://letterboxd.com/film/nosferatu/,I only watched this to understand the Spongebo...
2,Home Alone,home-alone,https://letterboxd.com/film/home-alone/,Young Patrick Bateman**do you think the wet ba...
3,The Holdovers,the-holdovers,https://letterboxd.com/film/the-holdovers/,There's a handshake in this that feels more ov...
4,Anora,anora,https://letterboxd.com/film/anora/,A terrifying tale of dating a mama’s boy**Sean...
...,...,...,...,...
3595,Puppy Love,puppy-love,https://letterboxd.com/film/puppy-love/,idk how much more of these i can take**not eno...
3596,Hancock,hancock,https://letterboxd.com/film/hancock/,First half of movie good second half is forget...
3597,Compartment No. 6,compartment-no-6,https://letterboxd.com/film/compartment-no-6/,How to ruin a party: invite a guy with a guita...
3598,Twenty Five Twenty One,twenty-five-twenty-one,https://letterboxd.com/film/twenty-five-twenty...,seriously minchae whose sperm are u**that endi...


In [8]:
# Supprimer les doublons dans df2 basés sur MOVIE_NAME, si nécessaire
df2_unique = df2.drop_duplicates(subset="MOVIE_NAME").set_index("MOVIE_NAME")

# Ajouter la colonne REVIEWS de df2 dans df1
df1["REVIEWS"] = df1["MOVIE_NAME"].map(df2_unique["REVIEWS"])


In [11]:
df1.to_csv("last_3000.csv", index=False)

## **Extraction du cast**

In [ ]:
import re

start_row = 3368
def fetch_movie_cast(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Échec lors de la récupération de {url}")
            return {"CAST": "Indisponible"}
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extraire le casting principal
        cast_section = soup.find('div', class_='cast-list')
        cast = [actor.text.strip() for actor in cast_section.find_all('a')] if cast_section else ["Indisponible"]

        return {"CAST": ", ".join(cast)}
    except Exception as e:
        print(f"Erreur pour {url}: {e}")
        return {"CAST": "Indisponible"}
for idx, row in df.iloc[start_row:].iterrows():  # Commence à partir de la ligne 1949
    if pd.notna(row["CAST"]) :  # Passer les films déjà traités
        continue
    
    details = fetch_movie_cast(row['URL_FILM'])

    df.loc[idx, "CAST"] = details["CAST"]

    # Sauvegarder les résultats après chaque itération pour éviter les pertes
    df.to_csv(r'C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\Llast3_det.csv', index=False)

## **Extraction du rating**

In [ ]:

# Charger le fichier CSV
file_path = r'C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\Llast3_det.csv'
df = pd.read_csv(file_path)

# Ajouter la colonne "RATING" si elle n'existe pas déjà
if "RATING_SUR_5" not in df.columns:
    df["RATING_SUR_5"] = None

# Fonction pour extraire le RATING
def fetch_movie_RATING(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extraire le RATING
        RATING_meta = soup.find('meta', {'name': 'twitter:data2'})
        RATING = RATING_meta['content'].strip() if RATING_meta else "Indisponible"
        return RATING
    except Exception as e:
        print(f"Erreur pour {url}: {e}")
        return "Indisponible"

# Définir la ligne de départ (start_row)
start_row = 2467  # Remplacer par l'index de départ voulu

# Traitement à partir de start_row
for idx, row in df.iloc[start_row:].iterrows():
    print(f"Traitement de l'index {idx}...")
    # Vérifier si "RATING" est déjà rempli
    if pd.notna(row["RATING_SUR_5"]):
        continue
    # Extraire le RATING
    rating = fetch_movie_RATING(row["URL_FILM"])
    df.loc[idx, "RATING_SUR_5"] = rating

# Sauvegarder les résultats dans le fichier CSV
df.to_csv(file_path, index=False)

print("Extraction terminée et sauvegardée dans le fichier CSV.")


Traitement de l'index 2467...


C:\Users\Aycha\AppData\Local\Temp\ipykernel_27992\4128496597.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.23 out of 5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, "RATING_SUR_5"] = rating


Traitement de l'index 2468...
Traitement de l'index 2469...
Traitement de l'index 2470...
Traitement de l'index 2471...
Traitement de l'index 2472...
Traitement de l'index 2473...
Traitement de l'index 2474...
Traitement de l'index 2475...
Traitement de l'index 2476...
Traitement de l'index 2477...
Traitement de l'index 2478...
Traitement de l'index 2479...
Traitement de l'index 2480...
Traitement de l'index 2481...
Traitement de l'index 2482...
Traitement de l'index 2483...
Traitement de l'index 2484...
Traitement de l'index 2485...
Traitement de l'index 2486...
Traitement de l'index 2487...
Traitement de l'index 2488...
Traitement de l'index 2489...
Traitement de l'index 2490...
Traitement de l'index 2491...
Traitement de l'index 2492...
Traitement de l'index 2493...
Traitement de l'index 2494...
Traitement de l'index 2495...
Traitement de l'index 2496...
Traitement de l'index 2497...
Traitement de l'index 2498...
Traitement de l'index 2499...
Traitement de l'index 2500...
Traitement

## **Extraction de la duration**

In [ ]:

# Charger le fichier CSV
file_path = r'C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\Llast3_det.csv'
df = pd.read_csv(file_path)

# Ajouter la colonne "Duration" si elle n'existe pas déjà
if "DURATION_MIN" not in df.columns:
    df["DURATION_MIN"] = "Inconnue"

# Fonction pour extraire la durée
def fetch_movie_duration(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Échec lors de la récupération de {url}")
            return {"DURATION_MIN": "Inconnue"}
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extraire la durée
        duration_div = soup.find('p', class_='text-link text-footer')
        if duration_div:
            # Utiliser une expression régulière pour trouver le nombre avant "mins"
            match = re.search(r"(\d+)\s?mins", duration_div.text)
            duration = f"{match.group(1)} minutes" if match else "Inconnue"
        else:
            duration = "Inconnue"

        return {"DURATION_MIN": duration}
    except Exception as e:
        print(f"Erreur pour {url}: {e}")
        return {"DURATION_MIN": "Inconnue"}

# Définir la ligne de départ (start_row)
start_row = 3087  # Remplacer par l'index de départ voulu

# Traitement à partir de start_row
for idx, row in df.iloc[start_row:].iterrows():
    print(f"Traitement de l'index {idx}...")
    if row["DURATION_MIN"] == "Inconnue":  # Passer les films déjà traités
        details = fetch_movie_duration(row['URL_FILM'])
        df.loc[idx, "DURATION_MIN"] = details["DURATION_MIN"]
        
    # Sauvegarder les résultats après chaque itération pour éviter les pertes
    df.to_csv(file_path, index=False)

print("Extraction terminée et sauvegardée dans le fichier CSV.")


Traitement de l'index 3087...
Traitement de l'index 3088...
Traitement de l'index 3089...
Traitement de l'index 3090...
Traitement de l'index 3091...
Traitement de l'index 3092...
Traitement de l'index 3093...
Traitement de l'index 3094...
Traitement de l'index 3095...
Traitement de l'index 3096...
Traitement de l'index 3097...
Traitement de l'index 3098...
Traitement de l'index 3099...
Traitement de l'index 3100...
Traitement de l'index 3101...
Traitement de l'index 3102...
Traitement de l'index 3103...
Traitement de l'index 3104...
Traitement de l'index 3105...
Traitement de l'index 3106...
Traitement de l'index 3107...
Traitement de l'index 3108...
Traitement de l'index 3109...
Traitement de l'index 3110...
Traitement de l'index 3111...
Traitement de l'index 3112...
Traitement de l'index 3113...
Traitement de l'index 3114...
Traitement de l'index 3115...
Traitement de l'index 3116...
Traitement de l'index 3117...
Traitement de l'index 3118...
Traitement de l'index 3119...
Traitement

## **Extraction du genre**

In [ ]:
# Fonction pour extraire les genres
def extract_genres(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lève une exception si le statut HTTP n'est pas 200
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Localiser le conteneur des genres
        genre_container = soup.find('div', class_='text-sluglist capitalize')
        if not genre_container:
            return None
        
        # Extraire les genres
        genres = [a.text for a in genre_container.find_all('a', class_='text-slug')]
        return '; '.join(genres)
    
    except Exception as e:
        print(f"Erreur lors du traitement de l'URL {url}: {e}")
        return None

# Charger le DataFrame
file_path = r'C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\Llast3_det.csv'
df = pd.read_csv(file_path)

# Ajouter la colonne "GENRES" si elle n'existe pas déjà
if "GENRES" not in df.columns:
    df["GENRES"] = None

# Traitement à partir d'un index spécifique
start_row = 3186  # Remplace par la ligne de départ souhaitée
for idx, row in df.iloc[start_row:].iterrows():
    print(f"Traitement de l'index {idx}...")

    # Vérifier si GENRES est déjà rempli
    if pd.notna(row["GENRES"]):
        continue

    # Extraire les genres
    genres = extract_genres(row["URL_FILM"])
    df.loc[idx, "GENRES"] = genres

    # Sauvegarder les résultats dans le CSV après chaque itération
    df.to_csv(file_path, index=False)

    # Pause pour éviter de surcharger le site
    time.sleep(1)

print("Extraction terminée.")


Traitement de l'index 3186...
Traitement de l'index 3187...
Traitement de l'index 3188...
Traitement de l'index 3189...
Traitement de l'index 3190...
Traitement de l'index 3191...
Traitement de l'index 3192...
Traitement de l'index 3193...
Erreur lors du traitement de l'URL https://letterboxd.com/film/scary-stories-to-tell-in-the-dark/: 404 Client Error: Not Found for url: https://letterboxd.com/film/scary-stories-to-tell-in-the-dark/
Traitement de l'index 3194...
Traitement de l'index 3195...
Traitement de l'index 3196...
Traitement de l'index 3197...
Traitement de l'index 3198...
Traitement de l'index 3199...
Traitement de l'index 3200...
Traitement de l'index 3201...
Traitement de l'index 3202...
Traitement de l'index 3203...
Traitement de l'index 3204...
Traitement de l'index 3205...
Traitement de l'index 3206...
Traitement de l'index 3207...
Traitement de l'index 3208...
Traitement de l'index 3209...
Traitement de l'index 3210...
Traitement de l'index 3211...
Traitement de l'index

## **Extraction des producers**

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Charger le fichier CSV
file_path = r'C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\Llast3_det.csv'
df = pd.read_csv(file_path)

# Ajouter la colonne "PRODUCERS" si elle n'existe pas déjà
if "PRODUCERS" not in df.columns:
    df["PRODUCERS"] = None

# Fonction pour extraire les producteurs
def extract_producers(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        producer_header = soup.find('span', class_='crewrole -full', string='Producers')
        if not producer_header:
            return None
        
        producer_container = producer_header.find_next('div', class_='text-sluglist')
        if not producer_container:
            return None
        
        producers = [a.text for a in producer_container.find_all('a', class_='text-slug')]
        return '; '.join(producers)
    
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du traitement de l'URL {url}: {e}")
        return None

# Définir la ligne de départ (start_row)
start_row = 2549  # Remplacer par l'index de départ voulu

# Traitement à partir de start_row
for idx, row in df.iloc[start_row:].iterrows():
    print(f"Traitement de l'index {idx}...")
    producers = extract_producers(row['URL_FILM'] + "crew/")
    producers_text = producers if producers else None
    
    df.at[idx, 'PRODUCERS'] = producers_text
    
    # Sauvegarder le DataFrame après chaque itération
    df.to_csv(file_path, index=False)

print("Extraction terminée et sauvegardée dans le fichier CSV.")


Traitement de l'index 2549...
Traitement de l'index 2550...
Traitement de l'index 2551...
Traitement de l'index 2552...
Traitement de l'index 2553...
Traitement de l'index 2554...
Erreur lors du traitement de l'URL https://letterboxd.com/film/a-i-artificial-intelligence/crew/: 404 Client Error: Not Found for url: https://letterboxd.com/film/a-i-artificial-intelligence/crew/
Traitement de l'index 2555...
Traitement de l'index 2556...
Traitement de l'index 2557...
Traitement de l'index 2558...
Traitement de l'index 2559...
Traitement de l'index 2560...
Traitement de l'index 2561...
Traitement de l'index 2562...
Traitement de l'index 2563...
Traitement de l'index 2564...
Traitement de l'index 2565...
Traitement de l'index 2566...
Traitement de l'index 2567...
Traitement de l'index 2568...
Traitement de l'index 2569...
Traitement de l'index 2570...
Traitement de l'index 2571...
Traitement de l'index 2572...
Traitement de l'index 2573...
Traitement de l'index 2574...
Traitement de l'index 2

In [8]:
df=pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\movies_with_details.csv")

In [10]:
df.to_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\movies_with_details.csv", index=False)

In [7]:
letterbox_df=pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\letter_box_movie\last.csv")

In [9]:
letterbox_df.head()

,Film Title,MOVIE_NAME,URL_FILM,Synopsis,Year,Cast,Duration,REVIEWS,RATING,GENRES,PRODUCERS,COUNTRIES
0,Wicked,wicked,https://letterboxd.com/film/wicked/,A cul-de-sac in an oppressive suburb becomes a...,1998,"Julia Stiles, William R. Moses, Patrick Muldoo...",88 minutes,My god.**This film is so unbelievably obnoxiou...,2.64 out of 5,Thriller,Frank Beddor; Greg Steinberg,NaN
1,Gladiator II,gladiator-ii,https://letterboxd.com/film/gladiator-ii/,Years after witnessing the death of the revere...,2024,"Paul Mescal, Denzel Washington, Pedro Pascal, ...",148 minutes,This review may contain spoilers. I can handle...,3.38 out of 5,History; Action; Adventure,Michael A. Pruss; Winston Azzopardi; David Fra...,NaN
2,Moana 2,moana-2,https://letterboxd.com/film/moana-2/,After receiving an unexpected call from her wa...,2024,"Auliʻi Cravalho, Dwayne Johnson, Hualālai Chun...",100 minutes,The water looked worse than it did in the orig...,2.88 out of 5,Comedy; Animation; Adventure; Family,Christina Chen; Yvett Merino Flores,Canada; USA
3,The Substance,the-substance,https://letterboxd.com/film/the-substance/,A fading celebrity decides to use a black mark...,2024,"Demi Moore, Margaret Qualley, Dennis Quaid, Ed...",141 minutes,mama a girl inside you**This review may contai...,3.86 out of 5,Horror; Science Fiction,Eric Fellner; Coralie Fargeat; Tim Bevan,France; UK
4,Our Little Secret,our-little-secret,https://letterboxd.com/film/our-little-secret/,After discovering their significant others are...,2024,"Lindsay Lohan, Ian Harding, Kristin Chenoweth,...",99 minutes,"Dating a much younger woman?… Close enough, we...",2.38 out of 5,Drama; Romance; Comedy,Lisa Gooding; Mike Elliott,NaN


In [ ]:
letterbox_df.rename(columns={'Duration': 'DURATION_PER_MIN'}, inplace=True)
letterbox_df.rename(columns={'Synopsis': 'SYNOPSIS'}, inplace=True)
letterbox_df.rename(columns={'Cast': 'CAST'}, inplace=True)
letterbox_df.rename(columns={'Year': 'YEAR'}, inplace=True)
letterbox_df.rename(columns={'Film Title': 'MOVIE_TITLE'}, inplace=True)

In [36]:


# Supprimer le mot "minutes" dans les valeurs
letterbox_df['DURATION_PER_MIN'] = letterbox_df['DURATION_PER_MIN'].str.replace(' minutes', '', regex=False)
letterbox_df['DURATION_PER_MIN'] = letterbox_df['DURATION_PER_MIN'].replace('Inconnue', None)
# Convertir les valeurs en entier si nécessaire
letterbox_df['DURATION_PER_MIN'] = pd.to_numeric(letterbox_df['DURATION_PER_MIN'], errors='coerce')
